In [ ]:
import sys
sys.path.append('../')

import pipeline.sql as plsql

import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
engine = plsql.create_engine('../config.yaml')
role = 'direccion_trabajo_inspections_write'

In [ ]:
qry = """set role {};
    select idfiscalizacion, matter_code, count(*)
    from cleaned.infracted_matters_updatedbook
    group by idfiscalizacion, matter_code;""".format(role)
tab = plsql.query(qry, engine)

In [ ]:
tab.matter_code = tab.matter_code.str.strip()
tab.matter_code[tab['matter_code'] == "-"] = ""
tab = tab[(tab.matter_code != '')]

In [ ]:
tab.head()

In [ ]:
qry = """set role {};
    select codigo as matter_code, statement
    from raw.tipificador;""".format(role)
tab_tipi = plsql.query(qry, engine)
tab_tipi.matter_code = tab_tipi.matter_code.str.strip()

In [ ]:
tab_tipi.head()

In [ ]:
tab_join = tab.merge(tab_tipi, left_on='matter_code', right_on='matter_code', how='left')

In [ ]:
tab_join.head()

In [ ]:
tab_basket = (tab
          .groupby(['idfiscalizacion', 'matter_code'])['count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('idfiscalizacion'))

In [ ]:
tab_basket.shape

In [ ]:
tab_basket.head()

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = tab_basket.applymap(encode_units)

In [ ]:
basket_sets.head()

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

In [ ]:
frequent_itemsets.sort_values('support', ascending= False)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules.head()

In [ ]:
rules.shape

In [ ]:
rules.antecedants

In [ ]:
#[1006-a, 1034-a, 1046-c]
qry = """set role {};
    select codigo, statement
    from raw.tipificador
    where codigo in ('1006-a', '1034-a', '1046-c');""".format(role)
plsql.query(qry, engine)